# Testing 
(Mainly locally, I don't know a method to get python notebook on vader5)


In [ ]:
import requests
import json 
from dotenv import load_dotenv
import os

base_url = 'http://localhost:8502/api/chatbot'

load_dotenv()
auth_key = os.getenv("AUTH_KEY")
auth_string = "&auth_key=" + auth_key + "&user_id=testing" # Only for testing
# In Version 1.6.1, the freva_config also needs to be set to a specific path. We won't be using this for now.
auth_string = auth_string + "&freva_config=" + "Cargo.toml" # Dummy value
auth_string = auth_string + "&user_id=" + "testing" # Dummy value

# As a test, ping the server
response = requests.get(base_url + "/ping")
print(response.text)

In [ ]:
# Also read the docs
response = requests.get(base_url + "/docs")
print(response.text)

In [ ]:
# Check which chatbots are available
response = requests.get(base_url + "/availablechatbots?" + auth_string)
print(response.json())



## Checking whether the code interpreter can be called

In [ ]:



# user_input = "This is a test request for your basic functionality. Please respond with (200 Ok) and exit."
# user_input = "This is a test request for your capabilities of using the code_interpreter tool. Please use the code_interpreter tool to run the following code exactly and only once: \"print('Hello, World!', flush=True)\"."
# user_input = "This is a test regarding your capabilities of using the code_interpreter tool and whether it supports matplotlib. Please use the code_interpreter tool to use matplotlib to visualize a sine wave from -2pi to +2pi."
# user_input = "This is a test regarding your capabilities of using the code_interpreter tool and whether it supports matplotlib. Please use the code_interpreter tool to run the following code: \"import numpy as np\nimport matplotlib.pyplot as plt\nimport matplotlib\nmatplotlib.use('agg')\nt = np.linspace(-2 * np.pi, 2 * np.pi, 100)\nsine_wave = np.sin(t)\nplt.figure(figsize=(10, 5))\nplt.plot(t, sine_wave, label='Sine Wave')\nplt.title('Sine Wave from -2π to 2π')\nplt.xlabel('Angle (radians)')\nplt.ylabel('Sine value')\nplt.axhline(0, color='black', linewidth=0.5, linestyle='--')\nplt.axvline(0, color='black', linewidth=0.5, linestyle='--')\nplt.grid()\nplt.legend()\nplt.show()\"."
# The below input doesn't use the corerct matplotlib backend, so it might do unexpected things.
user_input = "This is a test regarding your capabilities of using the code_interpreter tool and whether it supports matplotlib. Please use the code_interpreter tool to run the following code: \"import numpy as np\nimport matplotlib.pyplot as plt\nt = np.linspace(-2 * np.pi, 2 * np.pi, 100)\nsine_wave = np.sin(t)\nplt.figure(figsize=(10, 5))\nplt.plot(t, sine_wave, label='Sine Wave')\nplt.title('Sine Wave from -2π to 2π')\nplt.xlabel('Angle (radians)')\nplt.ylabel('Sine value')\nplt.axhline(0, color='black', linewidth=0.5, linestyle='--')\nplt.axvline(0, color='black', linewidth=0.5, linestyle='--')\nplt.grid()\nplt.legend()\nplt.show()\"."
url = base_url + '/streamresponse?input=' + user_input + "&chatbot=gpt-4o-mini" + auth_string # leaving out the thread_id spawns a new thread

response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.
complete_response = [] # The stream gets consumed when streamed, we'll store it here.

# Note that this way to cunsume the response chunks it and those chunks are surprisingly small, cutting up the response.
# A better way would be `for delta in response.iter_content(chunk_size=1024):`
for delta in response:
    print(delta)
    complete_response.append(delta.decode("utf-8"))

In [ ]:
# Some elements are cut off early if the chunk size is not specified or too small, so to reconstruct, we'll need to append all element that don't start with "{" to the previous element.
reconstructed_response = []
for i in range(len(complete_response)):
    if complete_response[i][0] == "{":
        reconstructed_response.append(complete_response[i])
    else:
        reconstructed_response[-1] += complete_response[i]

for delta in reconstructed_response:
    print(delta)

In [ ]:
# Now we can parse the response as JSON
import json
response_json = [json.loads(x) for x in reconstructed_response]
for element in response_json:
    print(element)

In [ ]:
# We can use this to easily extract only the Assistant's responses
assistant_responses = [x for x in response_json if x["variant"] == "Assistant"]
print("".join([x["content"] for x in assistant_responses])) # This will print the Assistant's responses in order.

In [ ]:
# Or the generated Code
code_responses = [x for x in response_json if x["variant"] == "Code"]
print("".join([x["content"][0] for x in code_responses]).replace("\\\n","\n")) # This will print the generated code in order.

In [ ]:
# Or all coded output
code_output_responses = [x["content"][0] for x in response_json if x["variant"] == "CodeOutput"]
for error in code_output_responses:
    print(error)

In [ ]:
# We can also extrace the image, if there is one
image_responses = [x["content"] for x in response_json if x["variant"] == "Image"]
# it's base64, so we can use IPython to display it
from IPython.display import display, Image
from base64 import b64decode
for image in image_responses:
    display(Image(data=b64decode(image), format='png')) # only for testing. This will display the image in the notebook.

In [ ]:
# Test the code_interpreter tool: This should return "Hello\nWorld\n!"
user_input = "Please use the code_interpreter tool to run the following code exactly and only once: \"print('Hello\\nWorld\\n!', flush=True)\"."
url = base_url + '/streamresponse?input=' + user_input + "&chatbot=gpt-4o-mini" + auth_string # leaving out the thread_id spawns a new thread
hello_world_response = requests.get(url, stream=True)

In [ ]:
# retrieve thread_id
hello_world_response_reconstructed = []
for delta in hello_world_response:
    if delta.decode("utf-8")[0] == "{":
        hello_world_response_reconstructed.append(delta.decode("utf-8"))
    else:
        hello_world_response_reconstructed[-1] += delta.decode("utf-8")


In [ ]:
import json
hello_world_response_json = [json.loads(x) for x in hello_world_response_reconstructed]
hw_thread_id = json.loads(hello_world_response_json[0]["content"])["thread_id"]
print(hw_thread_id)

# Also retrieve all variants seperately
hello_world_stream_output = [x["content"][0] for x in hello_world_response_json if x["variant"] == "CodeOutput"]
hello_world_stream_code = [x["content"][0] for x in hello_world_response_json if x["variant"] == "Code"]
hello_world_stream_assistant = [x["content"] for x in hello_world_response_json if x["variant"] == "Assistant"]
hello_world_stream_user = [x["content"] for x in hello_world_response_json if x["variant"] == "User"]

print("".join(hello_world_stream_output))
print("".join(hello_world_stream_code))
print("".join(hello_world_stream_assistant))
print("".join(hello_world_stream_user))


In [ ]:
# Now also retrieve the response from the getThread endpoint
url = base_url + '/getthread?thread_id=' + hw_thread_id + auth_string
hello_world_thread_response = requests.get(url)
hello_world_thread_json = hello_world_thread_response.json()
hello_world_thread_text = hello_world_thread_response.text
hello_world_thread_code_output = [x["content"][0] for x in hello_world_thread_json if x["variant"] == "CodeOutput"]
hello_world_thread_code = [x["content"][0] for x in hello_world_thread_json if x["variant"] == "Code"]
hello_world_thread_assistant = [x["content"] for x in hello_world_thread_json if x["variant"] == "Assistant"]
hello_world_thread_user = [x["content"] for x in hello_world_thread_json if x["variant"] == "User"]


print("".join(hello_world_thread_code_output))
print("".join(hello_world_thread_code))
print("".join(hello_world_thread_assistant))
print("".join(hello_world_thread_user))


In [ ]:
print(hello_world_thread_text)

### Test the get_thread endpoint

In [ ]:
# Generate a new thread asking it to add 2+2 in the code_interpreter
user_input = "Please add 2+2 in the code_interpreter tool."
url = base_url + '/streamresponse?input=' + user_input + "&chatbot=gpt-4o-mini" + auth_string # leaving out the thread_id spawns a new thread
response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.

In [ ]:
# To compare the output between streamresponse and getthread, output the json from getthread
recon_reponse = []
temp = response.text.split("}{")
for i in range(len(temp)):
    if i == 0:
        recon_reponse.append(temp[i] + "}")
    elif i == len(temp) - 1:
        recon_reponse.append("{" + temp[i])
    else:
        recon_reponse.append("{" + temp[i] + "}")
response_json = [json.loads(x) for x in recon_reponse]
response_json

In [ ]:
# Extract the thread_id
thread_id = json.loads(response_json[0]["content"])["thread_id"]
print(thread_id)

In [ ]:
# Use the thread_id to stream another response in the same thread
user_input = "Now please multiply that by 3."
url = base_url + '/streamresponse?input=' + user_input + '&thread_id=' + thread_id + "&chatbot=gpt-4o-mini" + auth_string
m3response = requests.get(url, stream=True)
m3complete_response = [] # The stream gets consumed when streamed, we'll store it here.

for delta in m3response:
    m3complete_response.append(delta.decode("utf-8"))

m3reconstructed_response = []
for i in range(len(m3complete_response)):
    if m3complete_response[i][0] == "{":
        m3reconstructed_response.append(m3complete_response[i])
    else:
        m3reconstructed_response[-1] += m3complete_response[i]

m3response_json = [json.loads(x) for x in m3reconstructed_response]


In [ ]:
m3code = [x for x in m3response_json if x["variant"] == "Code"]
print("".join([x["content"][0] for x in m3code]).replace("\\\n","\n")) # This will print the generated code in order.
m3assistant_responses = [x for x in m3response_json if x["variant"] == "Assistant"]
print("".join([x["content"] for x in m3assistant_responses])) # This will print the Assistant's responses in order.

In [ ]:
# Testing the get_thread endpoint
url = base_url + '/getthread?thread_id=' + thread_id + auth_string
getthreadresponse = requests.get(url)
getthreadresponse.text

In [ ]:
# comparing the text of both responses. Both should be almost identical, but the getthread response should have collapsed many of the variants and is also a list, not a stream. 
print(response.text)
print(getthreadresponse.text)
getthreadresponse.json() 

In [ ]:
# In the Version 1.6.3, the code_interpreter now has persistent storage, so we can test that here.
# We simply assign a variable in the code_interpreter and then retrieve it in a new thread.
user_input = "Please assign the value 42 to the variable x in the code_interpreter tool. After that, call the tool with the code \"print(x, flush=True)\", without assigning x again. It's a test for the presistance of data."
url = base_url + '/streamresponse?input=' + user_input + "&chatbot=gpt-4o-mini" + auth_string # leaving out the thread_id spawns a new thread
pers_response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.
pers_reconstructed_response = []
for delta in pers_response:
    if delta.decode("utf-8")[0] == "{":
        pers_reconstructed_response.append(delta.decode("utf-8"))
    else:
        pers_reconstructed_response[-1] += delta.decode("utf-8")

pers_response_json = [json.loads(x) for x in pers_reconstructed_response]

In [ ]:
print("".join([x["content"][0] for x in pers_response_json if x["variant"] == "Code"])) # Print the Code.
print("".join([x["content"][0] for x in pers_response_json if x["variant"] == "CodeOutput"])) # Print the CodeOutput.
print("".join([x["content"] for x in pers_response_json if x["variant"] == "Assistant"])) # This will print the Assistant's responses in order.
pers_thread_id = json.loads(pers_response_json[0]["content"])["thread_id"]
print(pers_thread_id)

In [ ]:
# In the Version 1.6.5, the capability to store xarray datasets between calls to the code_interpreter was added.
# This is a test to see if it works.
# We simply ask the LLM to generate a simple xarray dataset and then access it in a new code_interpreter call.
user_input = "Please generate a simple xarray dataset in the code_interpreter tool and print out the content. After that, call the tool with the code \"print(ds, flush=True)\", without generating the dataset again. It's a test for the presistance of data, specifically whether xarray Datasets also work."
url = base_url + '/streamresponse?input=' + user_input + "&chatbot=gpt-4o-mini" + auth_string # leaving out the thread_id spawns a new thread
xarray_response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.
xarray_reconstructed_response = []
for delta in xarray_response:
    if delta.decode("utf-8")[0] == "{":
        xarray_reconstructed_response.append(delta.decode("utf-8"))
    else:
        xarray_reconstructed_response[-1] += delta.decode("utf-8")

xarray_response_json = [json.loads(x) for x in xarray_reconstructed_response]


In [ ]:
# Print the results
print("".join([x["content"][0] for x in xarray_response_json if x["variant"] == "Code"])) # Print the Code.
print("".join([x["content"][0] for x in xarray_response_json if x["variant"] == "CodeOutput"])) # Print the CodeOutput.
print("".join([x["content"] for x in xarray_response_json if x["variant"] == "Assistant"])) # This will print the Assistant's responses in order.

In [ ]:
# In Version 1.7.1, the streamresponse endpoint was updated to allow for the setting of which chatbot to use. This tests whether it can be used.
# Stream a very simple request to llama instead of the default chatbot.
user_input = "This is a test request for your basic functionality. Please respond with (200 Ok) and exit."
url = base_url + '/streamresponse?input=' + user_input + "&chatbot=qwen2.5:3b" + auth_string # leaving out the thread_id spawns a new thread
llama_response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.
llama_complete_response = [] # The stream gets consumed when streamed, we'll store it here.

for delta in llama_response:
    llama_complete_response.append(delta.decode("utf-8"))

llama_reconstructed_response = []
for i in range(len(llama_complete_response)):
    if llama_complete_response[i][0] == "{":
        llama_reconstructed_response.append(llama_complete_response[i])
    else:
        llama_reconstructed_response[-1] += llama_complete_response[i]

In [ ]:
# Print out the response
llama_response_json = [json.loads(x) for x in llama_reconstructed_response]
print("".join([x["content"] for x in llama_response_json if x["variant"] == "Assistant"])) # This will print the Assistant's responses in order.

In [ ]:
# Stream a request to qwen2.5:3b to calculate 2938429834 * 234987234 using the code_interpreter tool.
user_input = "Please use the code_interpreter tool to run `print(2938429834 * 234987234)`. Make sure to adhere to the JSON format!"
url = base_url + '/streamresponse?input=' + user_input + "&chatbot=qwen2.5:3b" + auth_string # leaving out the thread_id spawns a new thread
qwen_response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.
qwen_complete_response = [] # The stream gets consumed when streamed, we'll store it here.

for delta in qwen_response:
    qwen_complete_response.append(delta.decode("utf-8"))
    print(delta.decode("utf-8"))

qwen_reconstructed_response = []
for i in range(len(qwen_complete_response)):
    if qwen_complete_response[i][0] == "{":
        qwen_reconstructed_response.append(qwen_complete_response[i])
    else:
        qwen_reconstructed_response[-1] += qwen_complete_response[i]


In [ ]:

# Print out the response
qwen_response_json = [json.loads(x) for x in qwen_reconstructed_response]
print("".join([x["content"] for x in qwen_response_json if x["variant"] == "Assistant"])) # This will print the Assistant's responses in order.
# for element in qwen_response_json:
#     print(element)

In [ ]:
# Test the heartbeat functionality introduced in 1.8.1 by sleeping for seven seconds in the code_interpreter tool.
user_input = "Please use the code_interpreter tool to run the following code: \"import time\ntime.sleep(7)\"."
url = base_url + '/streamresponse?input=' + user_input + "&chatbot=gpt-4o-mini" + auth_string # leaving out the thread_id spawns a new thread

response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.
heartbeat_response = [] # The stream gets consumed when streamed, we'll store it here.

for delta in response:
    print(delta)
    heartbeat_response.append(delta.decode("utf-8"))

In [ ]:
# Reconstruct the response
reconstructed_response = []
for i in range(len(heartbeat_response)):
    if heartbeat_response[i][0] == "{":
        reconstructed_response.append(heartbeat_response[i])
    else:
        reconstructed_response[-1] += heartbeat_response[i]

# Extract the ServerHint variant
server_hint = [x for x in reconstructed_response if json.loads(x)["variant"] == "ServerHint"]
print(server_hint)

# The second element of the list contains the heartbeat message
print(json.loads(server_hint[1])["content"])

In [ ]:
# Starting in 1.8.3, stopping a conversation also stops all tool calls. This is a test to see if it works.
# Because we can't be that sure that the tool call is running, we'll just have it wait 20 seconds, wait ourselves for 10 seconds and then stop the conversation.
user_input = "Please use the code_interpreter tool to run the following code: \"import time\ntime.sleep(20)\". I will stop the conversation after 10 seconds. This is a test for whether stopping the conversation also stops the tool call."
url = base_url + '/streamresponse?input=' + user_input + "&chatbot=gpt-4o-mini" + auth_string # leaving out the thread_id spawns a new thread

response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.
# We can't just wait for the tool call to finish, but we need to read the first response to get the thread_id
complete_response = [] # The stream gets consumed when streamed, we'll store it here.

reconstructed_response = []

first_run = True
for delta in response:
    decoded = delta.decode("utf-8")
    complete_response.append(decoded)
    print(decoded)

    # Only extract the thread_id from the first response
    if not first_run :
        continue

    first_run = False
    # We'll just expect that it didn't split the first response
    first_content = json.loads(decoded)["content"]
    

    # Extract the thread_id
    stopping_thread_id = json.loads(first_content)["thread_id"]
    print(stopping_thread_id)

    # Wait for 10 seconds
    import time
    time.sleep(10)

    # Stop the conversation
    url = base_url + '/stop?thread_id=' + stopping_thread_id + auth_string
    stop_response = requests.get(url)
    print(stop_response.text)

for i in range(len(complete_response)):
    if complete_response[i][0] == "{":
        reconstructed_response.append(complete_response[i])
    else:
        reconstructed_response[-1] += complete_response[i]

reconstructed_response




In [ ]:
# In Version 1.8.9, the conversation is not corrupted anymore if the tool call is stopped. This is a test to see if it works.
# (Continue the conversation with the ID from the previous test)
user_input = "Test: did the conversation continue after stopping the tool call?"
url = base_url + '/streamresponse?input=' + user_input + '&thread_id=' + stopping_thread_id + auth_string
response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.
complete_response = [] # The stream gets consumed when streamed, we'll store it here.

for delta in response:
    # print(delta)
    complete_response.append(delta.decode("utf-8"))

# Some elements are cut off early if the chunk size is not specified or too small, so to reconstruct, we'll need to append all element that don't start with "{" to the previous element.
reconstructed_response = []
for i in range(len(complete_response)):
    if complete_response[i][0] == "{":
        reconstructed_response.append(complete_response[i])
    else:
        reconstructed_response[-1] += complete_response[i]

for delta in reconstructed_response:
    print(delta)

In [ ]:
# Test the improved syntax error reporting in Version 1.8.4
user_input = "Please use the code_interpreter tool to run the following code: \"print('Hello World'\". This is a test for the improved syntax error reporting. If a hint containing the syntax error is returned, the test is successful."
url = base_url + '/streamresponse?input=' + user_input + "&chatbot=gpt-4o-mini" + auth_string # leaving out the thread_id spawns a new thread

response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.
syntax_error_response = [] # The stream gets consumed when streamed, we'll store it here.

for delta in response:
    print(delta)
    syntax_error_response.append(delta.decode("utf-8"))

# Reconstruct the response
reconstructed_response = []
for i in range(len(syntax_error_response)):
    if syntax_error_response[i][0] == "{":
        reconstructed_response.append(syntax_error_response[i])
    else:
        reconstructed_response[-1] += syntax_error_response[i]

# Print the reconstructed response
for delta in reconstructed_response:
    print(delta)

In [ ]:
# Extract the CodeOutput and Assistant variants
json_response = [json.loads(x) for x in reconstructed_response]
code_output = [x for x in json_response if x["variant"] == "CodeOutput"]
print("".join([x["content"][0] for x in code_output])) # This will print the generated code in order.
assistant_responses = [x for x in json_response if x["variant"] == "Assistant"]
print("".join([x["content"] for x in assistant_responses])) # This will print the Assistant's responses in order.

In [ ]:
# Test the 1.8.9 correction of the code_interpreter tool's handling of data:
# Does data persist across more than one call?
user_input = "This is a test on a corner case of the code_interpreter tool: variables don't seem to be stored if the code errors before the last line.\
To test this. Please run the following code: \"x = 42\nraise Exception('This is a test exception')\nprint('Padding for last-line-logic')\". After that, run the code \"print(x, flush=True)\" in a new block."
url = base_url + '/streamresponse?input=' + user_input + auth_string # leaving out the thread_id spawns a new thread

persistant_response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.
persistant_response_complete = [] # The stream gets consumed when streamed, we'll store it here.

for delta in persistant_response:
    print(delta)
    persistant_response_complete.append(delta.decode("utf-8"))

# Reconstruct the response
reconstructed_response = []
for i in range(len(persistant_response_complete)):
    if persistant_response_complete[i][0] == "{":
        reconstructed_response.append(persistant_response_complete[i])
    else:
        reconstructed_response[-1] += persistant_response_complete[i]

# Print the reconstructed response
for delta in reconstructed_response:
    print(delta)

# Extract the CodeOutput and Assistant variants
json_response = [json.loads(x) for x in reconstructed_response]
code = [x for x in json_response if x["variant"] == "Code"]
code_output = [x for x in json_response if x["variant"] == "CodeOutput"]
print("".join([x["content"][0] for x in code_output]))
print("".join([x["content"][0] for x in code])) # This will print the generated code in order.


In [ ]:
print("".join([x["content"] for x in json_response if x["variant"] == "Assistant"])) # This will print the Assistant's responses in order.
print()
print("\n".join([x["content"][0] for x in json_response if x["variant"] == "CodeOutput"])) # This will print the Code outputs
print()
print("".join([x["content"][0] for x in json_response if x["variant"] == "Code"])) 

In [ ]:
# In Version 1.8.13, support for o3-mini was added. This is a small test to see if it works.
# Note that support for deepseek-r1-70b was also added, but that requires the model to be running locally, which I can't test.
# Stream a very simple request to o3-mini instead of the default chatbot.

user_input = "This is a test request for your basic functionality. Please use the code interpreter to print 'Hello World!' and exit."
url = base_url + '/streamresponse?input=' + user_input + "&chatbot=o3-mini" + auth_string # leaving out the thread_id spawns a new thread
o3_response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.
o3_complete_response = [] # The stream gets consumed when streamed, we'll store it here.

for delta in o3_response:
    o3_complete_response.append(delta.decode("utf-8"))

o3_reconstructed_response = []
for i in range(len(o3_complete_response)):
    if o3_complete_response[i][0] == "{":
        o3_reconstructed_response.append(o3_complete_response[i])
    else:
        o3_reconstructed_response[-1] += o3_complete_response[i]

# Print out the response
o3_response_json = [json.loads(x) for x in o3_reconstructed_response]
print("".join([x["content"] for x in o3_response_json if x["variant"] == "Assistant"])) # This will print the Assistant's responses in order.

In [ ]:
for element in o3_response_json:
    print(element)

In [ ]:
# During cleanup (version 1.8.15 probably), support for input over headers was added. This is a test to see if it works.
# Note that this is not the same as the input over parameters in the streamresponse endpoint, which is still not supported.

user_input = "This is a test request for your basic functionality. Please respond with (200 Ok) and exit."
url = base_url + '/streamresponse?chatbot=gpt-4o-mini' # leaving out the thread_id spawns a new thread
headers = {
    'input': user_input,
    'Authorization': 'Bearer ' + auth_key, # Only for testing
    'X-freva-ConfigPath': 'Cargo.toml' # Dummy value
}
response = requests.get(url, headers=headers, stream=True) # The response can be streamed or gotten all at once.
complete_response = [] # The stream gets consumed when streamed, we'll store it here.
for delta in response:
    complete_response.append(delta.decode("utf-8"))
# Some elements are cut off early if the chunk size is not specified or too small, so to reconstruct, we'll need to append all element that don't start with "{" to the previous element.
reconstructed_response = []
for i in range(len(complete_response)):
    if complete_response[i][0] == "{":
        reconstructed_response.append(complete_response[i])
    else:
        reconstructed_response[-1] += complete_response[i]
for delta in reconstructed_response:
    print(delta)
# Print out the response
response_json = [json.loads(x) for x in reconstructed_response]
print("".join([x["content"] for x in response_json if x["variant"] == "Assistant"])) # This will print the Assistant's responses in order.